In [1]:
from azure.identity import AzureCliCredential
# New imports
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.sql.models import FirewallRule

# Module for connecting to SQL Server
import pyodbc

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [4]:
print(AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP)

5986c4d9-0694-4375-b863-e2da8648d7fc eastus azure-storage-group


In [2]:
credential = AzureCliCredential()
sql_client = SqlManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [5]:
sql_server = sql_client.servers.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    parameters={
        'location': DEFAULT_LOCATION,
        'version': '12.0',
        'administrator_login': 'benadmin',
        'administrator_login_password': 'testpassword123!'
    }
)
sql_server.wait()

In [6]:
for server in sql_client.servers.list():
    print(server.name)

bens-server


In [7]:
database = sql_client.databases.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    database_name='bens-db',
    parameters={
        'location': DEFAULT_LOCATION,
        'sample_name': 'AdventureWorksLT',
        'edition': 'GeneralPurpose',
        'compute_model': 'Serverless',
        'family': 'Gen5',
        'capacity': 2
    }
)
database.wait()

In [8]:
for db in sql_client.databases.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(db.name)

bens-db
master


In [42]:
SERVER = 'bens-server.database.windows.net'
DATABASE = 'bens-db'
USERNAME = 'benadmin'
PASSWORD = 'testpassword123!'
# Has to be installed
DRIVER = '{SQL Server}'

connection_string = f'DRIVER={DRIVER};PORT=1433;SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

In [44]:
firewall = sql_client.firewall_rules.create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    firewall_rule_name='JupyterNotebook',
    parameters={
        'start_ip_address': '157.45.165.10',
        'end_ip_address': '157.45.165.10'
    })


In [45]:
for fr in sql_client.firewall_rules.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(fr.name)

JupyterNotebook


In [46]:
print(pyodbc.drivers()) # this will list down all the drivers available in your local PC


['SQL Server']


In [47]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
connection_string

'DRIVER={SQL Server};PORT=1433;SERVER=bens-server.database.windows.net;DATABASE=bens-db;UID=benadmin;PWD=testpassword123!'

In [48]:
cursor.execute(
"""
select top (10) *
from [SalesLT].[Address]
"""
)
row = cursor.fetchone()
while row:
    print(str(row))
    row = cursor.fetchone()

(9, '8713 Yosemite Ct.', None, 'Bothell', 'Washington', 'United States', '98011', '268AF621-76D7-4C78-9441-144FD139821A', datetime.datetime(2006, 7, 1, 0, 0))
(11, '1318 Lasalle Street', None, 'Bothell', 'Washington', 'United States', '98011', '981B3303-ACA2-49C7-9A96-FB670785B269', datetime.datetime(2007, 4, 1, 0, 0))
(25, '9178 Jumping St.', None, 'Dallas', 'Texas', 'United States', '75201', 'C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6', datetime.datetime(2006, 9, 1, 0, 0))
(28, '9228 Via Del Sol', None, 'Phoenix', 'Arizona', 'United States', '85004', '12AE5EE1-FC3E-468B-9B92-3B970B169774', datetime.datetime(2005, 9, 1, 0, 0))
(32, '26910 Indela Road', None, 'Montreal', 'Quebec', 'Canada', 'H1Y 2H5', '84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D', datetime.datetime(2006, 8, 1, 0, 0))
(185, '2681 Eagle Peak', None, 'Bellevue', 'Washington', 'United States', '98004', '7BCCF442-2268-46CC-8472-14C44C14E98C', datetime.datetime(2006, 9, 1, 0, 0))
(297, '7943 Walnut Ave', None, 'Renton', 'Washington', 'Un

In [49]:
tables = cursor.tables(schema='SalesLT', tableType='TABLE')
for table in tables:
    print(table.table_name)

Address
Customer
CustomerAddress
Product
ProductCategory
ProductDescription
ProductModel
ProductModelProductDescription
SalesOrderDetail
SalesOrderHeader


In [51]:
# print(cursor)
print(cursor.rowcount)

-1


In [52]:
table_stats = cursor.statistics(table='Address', schema='SalesLT')
for row in table_stats:
    print(f'{row.column_name}')

None
AddressID
rowguid
AddressLine1
AddressLine2
City
StateProvince
PostalCode
CountryRegion
StateProvince


In [53]:
sql_server = sql_client.servers.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server'
)
sql_server.wait()